In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [10]:
df = pd.read_csv("C:/Users/admin/Desktop/研究生/Sem1/DSS5104/Project data/combined_B_&_C.csv")

In [3]:
print(df.head())

         date      price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0  2014-05-02   313000.0       3.0       1.50         1340      7912     1.5   
1  2014-05-02  2384000.0       5.0       2.50         3650      9050     2.0   
2  2014-05-02   342000.0       3.0       2.00         1930     11947     1.0   
3  2014-05-02   420000.0       3.0       2.25         2000      8030     1.0   
4  2014-05-02   550000.0       4.0       2.50         1940     10500     1.0   

   waterfront  view  condition  ...  city_grouped_SeaTac  \
0           0     0          3  ...                False   
1           0     4          5  ...                False   
2           0     0          4  ...                False   
3           0     0          4  ...                False   
4           0     0          4  ...                False   

   city_grouped_Seattle  city_grouped_Shoreline  city_grouped_Snoqualmie  \
0                 False                    True                    False   
1     

In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9102 entries, 0 to 9199
Data columns (total 69 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          9102 non-null   float64
 1   bedrooms                       9102 non-null   float64
 2   bathrooms                      9102 non-null   float64
 3   sqft_lot                       9102 non-null   int64  
 4   floors                         9102 non-null   float64
 5   waterfront                     9102 non-null   int64  
 6   view                           9102 non-null   int64  
 7   condition                      9102 non-null   int64  
 8   yr_built                       9102 non-null   int64  
 9   yr_renovated                   9102 non-null   int64  
 10  log_price                      9102 non-null   float64
 11  yr_after_renovated             9102 non-null   int64  
 12  yr_after_built                 9102 non-null   int64 

In [12]:
# Replace infinite values with NaN in the entire DataFrame
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [13]:
# 3. Define target and features
# We use log_price as the target and remove both 'price' and 'log_price' from the features
target_col = 'log_price'
df.dropna(subset=[target_col], inplace=True)  # ensure no missing target values

In [14]:
# 2. Data Cleaning
# Drop non-numeric columns that are not needed for regression
cols_to_drop = ['date', 'street', 'city', 'statezip', 'country', 'bedrooms_bin', 'bathrooms_bin', 'yr_built_bin', 'sqft_living', 'sqft_above', 'sqft_basement']
df.drop(columns=cols_to_drop, errors='ignore', inplace=True)

In [15]:
# 3. Definefeatures

X = df.drop(columns=['price', 'log_price'], errors='ignore')
y = df[target_col]

In [16]:
if 'yr_renovated_bin' in X.columns:
    X.drop(columns=['yr_renovated_bin'], inplace=True)

In [17]:
# One-hot encode the floors_bin column if it exists (since it has values like '1', '2', '3+')
if 'floors_bin' in df.columns:
    df = pd.get_dummies(df, columns=['floors_bin'], drop_first=True)

In [18]:
# Convert boolean columns to integers (0/1)
df = df.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

In [22]:
# Optional sanity check
print(X.select_dtypes(exclude='number').columns)

Index([], dtype='object')


In [20]:
# Just in case, Ensure X contains only numeric columns (drop any remaining non-numeric features)
X = X.select_dtypes(include=[np.number]).copy()

In [21]:
# 4. Split the data into training and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# 5. Setup Cross-Validation and scoring metrics
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
# Using sklearn's built-in scorers for MAE and RMSE.
scoring = {
    'MAE': 'neg_mean_absolute_error', 
    'RMSE': 'neg_root_mean_squared_error'
}

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, RegressorMixin
import statsmodels.api as sm

In [30]:
# Custom wrapper for statsmodels OLS to be used in scikit-learn pipelines
class OLSWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, add_constant=True):
        self.add_constant = add_constant

    def fit(self, X, y):
        # Add intercept if required (statsmodels does not add it automatically)
        if self.add_constant:
            X = sm.add_constant(X)
        self.model_ = sm.OLS(y, X).fit()
        return self

    def predict(self, X):
        if self.add_constant:
            # Make sure we add the constant if needed during prediction
            # 'has_constant' parameter ensures that a constant column isn’t added twice if it already exists.
            X = sm.add_constant(X, has_constant='add')
        return self.model_.predict(X)

    def summary(self):
        return self.model_.summary()

In [31]:
from sklearn.impute import SimpleImputer
# 6. Build Pipelines for the three models

# (Ordinary Linear Regression)
# ols_pipeline = Pipeline([
#    ('imputer', SimpleImputer(strategy='median')),
#    ('scaler', StandardScaler()),
#    ('reg', LinearRegression())
#])

# Baseline: OLS (Ordinary Least Squares) using our custom OLSWrapper
ols_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('reg', OLSWrapper())  # Our custom estimator wrapping statsmodels.OLS
])

# Ridge Regression with built-in cross-validation for alpha tuning
ridge_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('reg', RidgeCV(alphas=[0.01, 0.1, 1, 10, 100], cv=5))
])

# Lasso Regression with built-in cross-validation for alpha tuning
lasso_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('reg', LassoCV(alphas=[0.01, 0.1, 1, 10, 100], cv=5, random_state=42, max_iter=10000))
])

In [32]:
# 7. Evaluation function: performs CV and computes test set errors
def evaluate_model(pipeline, X_train, y_train, X_test, y_test, scoring, cv):
    # Cross-validation on training set
    cv_results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring=scoring, return_train_score=True)
    
    # Fit the pipeline on the full training set
    pipeline.fit(X_train, y_train)
    # Predict on the test set
    y_pred_test = pipeline.predict(X_test)
    
    # Compute test errors
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    # Convert negative scores to positive values
    train_mae = -np.mean(cv_results['train_MAE'])
    train_rmse = -np.mean(cv_results['train_RMSE'])
    cv_mae = -np.mean(cv_results['test_MAE'])
    cv_rmse = -np.mean(cv_results['test_RMSE'])
    
    return {
        'Train MAE': train_mae,
        'Train RMSE': train_rmse,
        'CV MAE': cv_mae,
        'CV RMSE': cv_rmse,
        'Test MAE': test_mae,
        'Test RMSE': test_rmse
    }

In [33]:
# 8. Evaluate each model
results_ols   = evaluate_model(ols_pipeline,   X_train, y_train, X_test, y_test, scoring, cv)
results_ridge = evaluate_model(ridge_pipeline, X_train, y_train, X_test, y_test, scoring, cv)
results_lasso = evaluate_model(lasso_pipeline, X_train, y_train, X_test, y_test, scoring, cv)

In [34]:
# 9. Compile results into a summary table
results_df = pd.DataFrame({
    'Model': ['OLS', 'Ridge', 'Lasso'],
    'Train MAE': [results_ols['Train MAE'], results_ridge['Train MAE'], results_lasso['Train MAE']],
    'CV MAE': [results_ols['CV MAE'], results_ridge['CV MAE'], results_lasso['CV MAE']],
    'Test MAE': [results_ols['Test MAE'], results_ridge['Test MAE'], results_lasso['Test MAE']],
    'Train RMSE': [results_ols['Train RMSE'], results_ridge['Train RMSE'], results_lasso['Train RMSE']],
    'CV RMSE': [results_ols['CV RMSE'], results_ridge['CV RMSE'], results_lasso['CV RMSE']],
    'Test RMSE': [results_ols['Test RMSE'], results_ridge['Test RMSE'], results_lasso['Test RMSE']]
})

print("Model Evaluation Results:\n", results_df)

Model Evaluation Results:
    Model  Train MAE    CV MAE  Test MAE  Train RMSE   CV RMSE  Test RMSE
0    OLS   0.161159  0.165279  0.172883    0.239377  0.363121   0.259911
1  Ridge   0.161232  0.164905  0.173314    0.238990  0.357968   0.259486
2  Lasso   0.209647  0.213785  0.233273    0.291992  0.421131   0.308900


In [35]:
# 10. Save the evaluation table to a CSV file
results_df.to_csv('linear_model_evaluation.csv', index=False)
print("Evaluation results saved to 'linear_model_evaluation.csv'")

Evaluation results saved to 'linear_model_evaluation.csv'


In [36]:
from sklearn.metrics import r2_score

# Predict on the test set for each model using the already fitted pipelines
y_pred_ols   = ols_pipeline.predict(X_test)
y_pred_ridge = ridge_pipeline.predict(X_test)
y_pred_lasso = lasso_pipeline.predict(X_test)

# Calculate R^2 score for each model
r2_ols   = r2_score(y_test, y_pred_ols)
r2_ridge = r2_score(y_test, y_pred_ridge)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Print the previously calculated evaluation metrics along with the R^2 score
print("Detailed Model Evaluation Results:")
print("OLS Results:")
print(f"  Train MAE: {results_ols['Train MAE']:.4f}")
print(f"  Train RMSE: {results_ols['Train RMSE']:.4f}")
print(f"  CV MAE: {results_ols['CV MAE']:.4f}")
print(f"  CV RMSE: {results_ols['CV RMSE']:.4f}")
print(f"  Test MAE: {results_ols['Test MAE']:.4f}")
print(f"  Test RMSE: {results_ols['Test RMSE']:.4f}")
print(f"  Test R^2: {r2_ols:.4f}\n")

print("Ridge Results:")
print(f"  Train MAE: {results_ridge['Train MAE']:.4f}")
print(f"  Train RMSE: {results_ridge['Train RMSE']:.4f}")
print(f"  CV MAE: {results_ridge['CV MAE']:.4f}")
print(f"  CV RMSE: {results_ridge['CV RMSE']:.4f}")
print(f"  Test MAE: {results_ridge['Test MAE']:.4f}")
print(f"  Test RMSE: {results_ridge['Test RMSE']:.4f}")
print(f"  Test R^2: {r2_ridge:.4f}\n")

print("Lasso Results:")
print(f"  Train MAE: {results_lasso['Train MAE']:.4f}")
print(f"  Train RMSE: {results_lasso['Train RMSE']:.4f}")
print(f"  CV MAE: {results_lasso['CV MAE']:.4f}")
print(f"  CV RMSE: {results_lasso['CV RMSE']:.4f}")
print(f"  Test MAE: {results_lasso['Test MAE']:.4f}")
print(f"  Test RMSE: {results_lasso['Test RMSE']:.4f}")
print(f"  Test R^2: {r2_lasso:.4f}")


Detailed Model Evaluation Results:
OLS Results:
  Train MAE: 0.1612
  Train RMSE: 0.2394
  CV MAE: 0.1653
  CV RMSE: 0.3631
  Test MAE: 0.1729
  Test RMSE: 0.2599
  Test R^2: 0.7611

Ridge Results:
  Train MAE: 0.1612
  Train RMSE: 0.2390
  CV MAE: 0.1649
  CV RMSE: 0.3580
  Test MAE: 0.1733
  Test RMSE: 0.2595
  Test R^2: 0.7619

Lasso Results:
  Train MAE: 0.2096
  Train RMSE: 0.2920
  CV MAE: 0.2138
  CV RMSE: 0.4211
  Test MAE: 0.2333
  Test RMSE: 0.3089
  Test R^2: 0.6625


In [37]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9102 entries, 0 to 9199
Data columns (total 69 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          9102 non-null   float64
 1   bedrooms                       9102 non-null   float64
 2   bathrooms                      9102 non-null   float64
 3   sqft_lot                       9102 non-null   int64  
 4   floors                         9102 non-null   float64
 5   waterfront                     9102 non-null   int64  
 6   view                           9102 non-null   int64  
 7   condition                      9102 non-null   int64  
 8   yr_built                       9102 non-null   int64  
 9   yr_renovated                   9102 non-null   int64  
 10  log_price                      9102 non-null   float64
 11  yr_after_renovated             9102 non-null   int64  
 12  yr_after_built                 9102 non-null   int64 

In [38]:
# Fit the pipeline on the training data
ols_pipeline.fit(X_train, y_train)

# Access the statsmodels OLS results via the custom wrapper
ols_results = ols_pipeline.named_steps['reg']

# Print the full regression output
print(ols_results.summary())


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     1278.
Date:                Tue, 15 Apr 2025   Prob (F-statistic):               0.00
Time:                        12:37:04   Log-Likelihood:                -151.12
No. Observations:                7281   AIC:                             348.2
Df Residuals:                    7258   BIC:                             506.8
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.0661      0.003   4504.603      0.0

In [40]:
# Fit the ridge pipeline on my training data
ridge_pipeline.fit(X_train, y_train)

# Access the trained ridge regressor from the pipeline
ridge_reg = ridge_pipeline.named_steps['reg']

# Retrieve coefficients and intercept
ridge_coef = ridge_reg.coef_
ridge_intercept = ridge_reg.intercept_

# Create a summary table with feature names
# Get feature names from X_train (assumes X_train is a DataFrame)
features = X_train.columns
ridge_summary = pd.DataFrame({
    'Feature': ['Intercept'] + list(features),
    'Coefficient': [ridge_intercept] + list(ridge_coef)
})

print("Ridge Regression Summary:")
print(ridge_summary)


Ridge Regression Summary:
               Feature  Coefficient
0            Intercept    13.066131
1             bedrooms    -0.033703
2            bathrooms     0.041230
3             sqft_lot     0.004708
4               floors     0.036089
5           waterfront     0.011052
6                 view     0.045268
7            condition     0.030034
8             yr_built    -0.020480
9         yr_renovated    -0.000526
10  yr_after_renovated    -0.006805
11      yr_after_built     0.020480
12      price_per_sqft     0.120300
13    basement_present     0.039438
14        if_renovated     0.006613
15       if_waterfront     0.011052
16      basement_ratio     0.022949
17    sqft_above_ratio    -0.004753
18               rooms     0.001178
19     log_sqft_living     0.144977
20      log_sqft_above     0.135275
21   log_sqft_basement    -0.013919
22             zipcode     0.043058
23      city_avg_price     0.075589
24   zipcode_avg_price     0.159842
25       knn_avg_price     0.034454


In [41]:
# Fit the lasso pipeline on my training data
lasso_pipeline.fit(X_train, y_train)

# Access the trained lasso regressor from the pipeline
lasso_reg = lasso_pipeline.named_steps['reg']

# Retrieve coefficients and intercept
lasso_coef = lasso_reg.coef_
lasso_intercept = lasso_reg.intercept_

# Create a summary table with feature names
features = X_train.columns
lasso_summary = pd.DataFrame({
    'Feature': ['Intercept'] + list(features),
    'Coefficient': [lasso_intercept] + list(lasso_coef)
})

print("Lasso Regression Summary:")
print(lasso_summary)


Lasso Regression Summary:
               Feature  Coefficient
0            Intercept    13.066131
1             bedrooms     0.000000
2            bathrooms     0.000000
3             sqft_lot     0.000000
4               floors     0.000000
5           waterfront     0.000000
6                 view     0.000000
7            condition     0.000000
8             yr_built    -0.000000
9         yr_renovated    -0.000000
10  yr_after_renovated     0.000000
11      yr_after_built     0.000000
12      price_per_sqft     0.032557
13    basement_present     0.000000
14        if_renovated    -0.000000
15       if_waterfront     0.000000
16      basement_ratio     0.000000
17    sqft_above_ratio    -0.000000
18               rooms     0.000000
19     log_sqft_living     0.182775
20      log_sqft_above     0.000000
21   log_sqft_basement     0.000000
22             zipcode     0.000000
23      city_avg_price     0.001875
24   zipcode_avg_price     0.165406
25       knn_avg_price     0.062225
